In [3]:
import os
import torch
import random
import math
from torch import nn
from torch.nn.modules import activation
import torch.nn.functional as F
import torchvision
import shutil
import numpy as np
from PIL import Image
import time
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix

C:\Users\GillA\Anaconda3\envs\Aseem\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
class AttentionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AttentionEncoder, self).__init__()
        
        # Define Queries, Keys, and Values
        self.linear_q = nn.Linear(input_size, hidden_size)
        self.linear_k = nn.Linear(input_size, hidden_size)
        self.linear_v = nn.Linear(input_size, hidden_size)
        self.linear_x = nn.Linear(input_size, hidden_size)
        
        # Multi Head Attention
        self.attention = nn.MultiheadAttention(hidden_size, num_heads=1, batch_first=True)
    def forward(self, x):
        
        # Obtain Queries, Keys, and Values
        q, k, v = self.linear_q(x), self.linear_k(x), self.linear_v(x)
        
        # Compute Attention Matrix
        x = self.attention(q,k,v)
        return x

class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters, hidden_size):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        # Define MaxPool First
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.conv3 = nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 25, padding = 'same')
        
        # Define Multi-Head Attention
        self.hidden_size = hidden_size
        self.attention = AttentionEncoder(input_size  = 3840, hidden_size = hidden_size)
        self.fc1 = nn.Linear(in_features = hidden_size, out_features = 1024)
        self.fc2 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        hidden_size = self.hidden_size
        ### Feature Learning Head
        
        # Ensure Batch is first and data type is correct
        x = torch.moveaxis(x,2,1)
        x = torch.tensor(x, dtype=torch.float32)

        # Parrallel Convolutions
        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
        # Concatenate
        x = torch.cat((x_S,x_M,x_L,x_O),1)
        
        # Post Concatenation
        x = self.MPool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
                
        # Flattening
        x = x.view(-1,3840)
        
        # Compute Attention
        x = self.attention(x)
        
        # Flattening
        x = x[0]
        x = x.view(-1,hidden_size)
        
        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.fc2(x)
        
        return (x)